In [2]:
# --- Extract the archive.zip file ---
import os
import zipfile

zip_path = '/content/archive (1).zip'
extract_path = '/content'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Extraction complete. Data is available in:", os.path.join(extract_path, "Data"))

import time
import copy
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import timm

# Set device (GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# Define data directories. The extracted data is in /content/Data.
data_dir = "/content/Data"  # Contains 'train', 'valid', and 'test' folders

# Define data transformations.
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    "valid": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
}

# Load the dataset using ImageFolder.
image_datasets = {
    phase: datasets.ImageFolder(os.path.join(data_dir, phase), data_transforms[phase])
    for phase in ["train", "valid", "test"]
}

# Adjusted DataLoader: reduced batch_size and num_workers.
dataloaders = {
    phase: DataLoader(image_datasets[phase], batch_size=16, shuffle=(phase=="train"), num_workers=0, pin_memory=True)
    for phase in ["train", "valid", "test"]
}

dataset_sizes = {phase: len(image_datasets[phase]) for phase in ["train", "valid", "test"]}
class_names = image_datasets["train"].classes
print("Classes:", class_names)

# Use a smaller ViT model variant to reduce memory usage:
num_classes = len(class_names)
# use 'vit_tiny_patch16_224' instead of 'vit_base_patch16_224'
model = timm.create_model("vit_tiny_patch16_224", pretrained=True, num_classes=num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch + 1, num_epochs))
        print("-" * 10)
        for phase in ["train", "valid"]:
            model.train() if phase == "train" else model.eval()
            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in tqdm(dataloaders[phase], desc=f"{phase}"):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == "train":
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            if phase == "valid" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    time_elapsed = time.time() - since
    print("Training complete in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60))
    print("Best valid Acc: {:.4f}".format(best_acc))
    model.load_state_dict(best_model_wts)
    return model

num_epochs = 10
model = train_model(model, criterion, optimizer, scheduler, num_epochs=num_epochs)

def evaluate_model(model, dataloader, class_names):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Testing"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))
    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

evaluate_model(model, dataloaders["test"], class_names)

Extraction complete. Data is available in: /content/Data
Using device: cuda:0
Classes: ['adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa', 'normal', 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

Epoch 1/10
----------


train: 100%|██████████| 39/39 [00:09<00:00,  4.01it/s]


...

train Loss: 0.0627 Acc: 0.9853


valid: 100%|██████████| 5/5 [00:00<00:00,  9.25it/s]


valid Loss: 0.5272 Acc: 0.8194

Training complete in 1m 31s
Best valid Acc: 0.8194


Testing: 100%|██████████| 20/20 [00:02<00:00,  7.80it/s]


Classification Report:
                                                  precision    recall  f1-score   support

      adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib       0.80      0.84      0.82       120
   large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa       0.60      0.96      0.74        51
                                          normal       0.93      0.96      0.95        54
squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa       0.92      0.52      0.67        90

                                        accuracy                           0.79       315
                                       macro avg       0.81      0.82      0.79       315
                                    weighted avg       0.82      0.79      0.78       315

Confusion Matrix:
[[101  15   0   4]
 [  2  49   0   0]
 [  1   1  52   0]
 [ 22  17   4  47]]


In [3]:
import numpy as np

In [4]:
cm = np.array([[100,16,0,4],
 [  2  ,49,0,0],
 [0,1,53, 0],
 [ 40,7,0,43]])
# Compute per-class accuracy (in percentage)
per_class_accuracy = 100.0 * np.diag(cm) / np.sum(cm, axis=1)

# If you have class names in a list (optional), for example:
class_names = ['adenocarcinoma', 'Large Cell Carcinoma', 'Normal', 'Squamous Cell Carcinoma']

# Print the per-class accuracies
for i, acc in enumerate(per_class_accuracy):
    if i < len(class_names):
        print(f"{class_names[i]}: {acc:.2f}% accuracy")
    else:
        print(f"Class {i}: {acc:.2f}% accuracy")

adenocarcinoma: 83.33% accuracy
Large Cell Carcinoma: 96.08% accuracy
Normal: 98.15% accuracy
Squamous Cell Carcinoma: 47.78% accuracy


In [5]:
# Save the model's state dictionary to a file.
model_save_path = '/content/trained_model.pth'
torch.save(model.state_dict(), model_save_path)
print("Model saved to:", model_save_path)


Model saved to: /content/trained_model.pth


In [6]:
# To load the saved model later:
model = timm.create_model("vit_tiny_patch16_224", pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load(model_save_path, map_location=device))
model.to(device)
model.eval()  # Set to evaluation mode


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
